In [1]:
import pyodbc
import pandas as pd
from datetime import datetime, date, timedelta
import requests
import concurrent.futures
import time
import random
import logging

# 設置日誌
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='stock_update.log',
    filemode='a'
)
logger = logging.getLogger('StockUpdater')

class FastStockUpdater:
    def __init__(self, server="IDEAPAD701\\MSSQLSERVER01", database="stock"):
        """初始化更新器"""
        self.connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
        self.session = requests.Session()
    
    def connect_db(self):
        """建立數據庫連接"""
        return pyodbc.connect(self.connection_string)
    
    def get_stock_tables(self):
        """獲取所有股票表名"""
        conn = self.connect_db()
        try:
            cursor = conn.cursor()
            # 獲取所有表，不做過濾
            cursor.execute("""
            SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
            """)
            return [row[0] for row in cursor.fetchall()]
        finally:
            conn.close()
    
    def check_missing_data(self, stock_table, start_date):
        """檢查缺失的日期數據"""
        conn = self.connect_db()
        try:
            cursor = conn.cursor()
            try:
                cursor.execute(f"""
                SELECT [Date] FROM [stock].[dbo].[{stock_table}]
                WHERE [Date] >= '{start_date}'
                ORDER BY [Date]
                """)
                
                # 獲取實際日期
                actual_dates = {row[0].date() if hasattr(row[0], 'date') else row[0] for row in cursor.fetchall()}
                
                # 計算應該有的交易日 (排除週末)
                expected_dates = set()
                start = datetime.strptime(start_date, '%Y-%m-%d')
                end = datetime.now()
                current = start
                one_day = timedelta(days=1)
                
                while current <= end:
                    if current.weekday() < 5:  # 排除週末
                        expected_dates.add(current.date())
                    current = current + one_day
                
                # 返回缺失的日期
                return sorted(list(expected_dates - actual_dates))
            except Exception:
                # 如果執行失敗，可能是表結構不符合股票數據表格式
                logger.warning(f"表 {stock_table} 可能不是股票數據表或結構不符")
                return []
        finally:
            conn.close()
    
    def get_stock_number(self, stock_table):
        """嘗試從表名找出股票代碼"""
        # 處理特殊情況: 直接從表名提取數字部分
        if any(c.isdigit() for c in stock_table):
            digits = ''.join(c for c in stock_table if c.isdigit())
            if digits:
                logger.info(f"從表名 {stock_table} 直接提取股票代碼: {digits}")
                return digits
        
        # 從資料庫中獲取可能的代碼映射
        conn = self.connect_db()
        try:
            cursor = conn.cursor()
            
            # 檢查表是否存在
            try:
                # 檢查特定表是否存在
                cursor.execute("""
                SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES 
                WHERE TABLE_NAME = 'StockNameMapping' AND TABLE_TYPE = 'BASE TABLE'
                """)
                
                table_exists = cursor.fetchone()[0] > 0
                
                # 如果表存在，嘗試查詢
                if table_exists:
                    cursor.execute("SELECT code FROM dbo.StockNameMapping WHERE name = ?", stock_table)
                    result = cursor.fetchone()
                    if result:
                        return result[0]
            except Exception as e:
                logger.error(f"查詢映射表失敗: {str(e)}")
            
            # 嘗試從其他表的已有數據判斷
            try:
                cursor.execute(f"""
                SELECT TOP 1 s2.TABLE_NAME
                FROM INFORMATION_SCHEMA.COLUMNS c1
                JOIN INFORMATION_SCHEMA.COLUMNS c2 ON c1.COLUMN_NAME = c2.COLUMN_NAME
                JOIN INFORMATION_SCHEMA.TABLES s1 ON c1.TABLE_NAME = s1.TABLE_NAME
                JOIN INFORMATION_SCHEMA.TABLES s2 ON c2.TABLE_NAME = s2.TABLE_NAME
                WHERE s1.TABLE_NAME = '{stock_table}'
                AND s2.TABLE_NAME LIKE '%TW'
                AND s1.TABLE_TYPE = 'BASE TABLE' AND s2.TABLE_TYPE = 'BASE TABLE'
                AND c1.DATA_TYPE = c2.DATA_TYPE
                """)
                
                result = cursor.fetchone()
                if result:
                    # 從TW結尾的表名中提取代碼
                    tw_table = result[0]
                    code = ''.join(c for c in tw_table if c.isdigit())
                    if code:
                        return code
            except Exception as e:
                logger.error(f"關聯查詢失敗: {str(e)}")
            
            # 最後嘗試從表名提取代碼部分
            return ''.join(c for c in stock_table if c.isdigit())
        finally:
            conn.close()
    
    def crawl_stock_data(self, stock_number, year, month, max_retries=3):
        """從台灣證交所爬取月度股票數據，含重試機制"""
        # 移除可能的中文和TW後綴
        stock_number = ''.join([c for c in stock_number if c.isdigit()])
        
        if not stock_number:
            return None
        
        for attempt in range(max_retries):
            try:
                # 隨機化請求頭，降低被封風險
                headers = {
                    'User-Agent': random.choice([
                        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/91.0.4472.124',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Safari/605.1.15',
                        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
                        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59'
                    ]),
                    'Referer': 'https://www.twse.com.tw/zh/'
                }
                
                # 構建URL
                url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={year}{month:02d}01&stockNo={stock_number}"
                
                response = self.session.get(url, headers=headers, timeout=5)  # 縮短超時時間
                
                if response.status_code != 200:
                    logger.warning(f"請求失敗: HTTP {response.status_code}, 嘗試 {attempt+1}/{max_retries}")
                    continue
                    
                data = response.json()
                
                if not data.get('data'):
                    logger.warning(f"未找到數據: {stock_number} {year}-{month}")
                    return None
                    
                # 轉換為DataFrame
                df = pd.DataFrame(data['data'], columns=[
                    'Date', 'Volume', 'Volume_Value', 'Open', 
                    'High', 'Low', 'Close', 'Change', 'Trades'
                ])
                
                # 轉換日期格式
                df['Date'] = df['Date'].apply(lambda x: f"{int(x.split('/')[0])+1911}-{x.split('/')[1].zfill(2)}-{x.split('/')[2].zfill(2)}")
                    
                # 轉換數值
                for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
                    df[col] = df[col].replace('--', '0').str.replace(',', '').astype(float)
                    
                return df
                
            except Exception as e:
                logger.error(f"爬取失敗 {stock_number} {year}-{month}: {str(e)}, 嘗試 {attempt+1}/{max_retries}")
                # 不休息，立即重試
                continue
        
        return None  # 所有重試都失敗

    def insert_stock_data(self, stock_table, data):
        """插入股票數據到數據庫"""
        if data is None or data.empty:
            return 0
            
        conn = self.connect_db()
        inserted = 0
        
        try:
            cursor = conn.cursor()
            
            for _, row in data.iterrows():
                try:
                    date_str = row['Date']
                    
                    # 檢查數值
                    if any(pd.isna(row[col]) for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
                        continue
                        
                    # 檢查記錄是否已存在
                    cursor.execute(f"SELECT 1 FROM [stock].[dbo].[{stock_table}] WHERE [Date] = ?", date_str)
                    if cursor.fetchone():
                        continue
                        
                    # 插入新記錄
                    cursor.execute(f"""
                    INSERT INTO [stock].[dbo].[{stock_table}] 
                    ([Date], [Open], [High], [Low], [Close], [Volume], [created_at])
                    VALUES (?, ?, ?, ?, ?, ?, GETDATE())
                    """, 
                    date_str, float(row['Open']), float(row['High']), 
                    float(row['Low']), float(row['Close']), float(row['Volume']))
                    
                    inserted += 1
                        
                except Exception as e:
                    logger.error(f"插入記錄失敗 {date_str}: {str(e)}")
                    continue
                    
            conn.commit()
            return inserted
            
        except Exception as e:
            logger.error(f"批量插入失敗 {stock_table}: {str(e)}")
            return 0
        finally:
            conn.close()
    
    def process_stock_table(self, stock_table, start_date):
        """處理單個股票表的數據更新"""
        try:
            logger.info(f"處理表: {stock_table}")
            print(f"處理表: {stock_table}")  # 同時在控制台顯示
            
            missing_dates = self.check_missing_data(stock_table, start_date)
            
            if not missing_dates:
                logger.info(f"{stock_table}: 數據完整")
                return 0
                
            logger.info(f"{stock_table}: 缺失 {len(missing_dates)} 筆數據")
            
            # 找出股票代碼
            stock_number = self.get_stock_number(stock_table)
            logger.info(f"使用股票代碼: {stock_number}")
            
            # 按月份分組
            month_groups = {}
            for d in missing_dates:
                key = (d.year, d.month)
                if key not in month_groups:
                    month_groups[key] = []
                month_groups[key].append(d)
            
            total_inserted = 0
            
            # 逐月處理
            for (year, month), dates in month_groups.items():
                logger.info(f"處理 {year}-{month} 月數據...")
                
                # 爬取月度數據
                monthly_data = self.crawl_stock_data(stock_number, year, month)
                
                if monthly_data is not None:
                    # 過濾出缺失日期的數據
                    target_dates = [d.strftime('%Y-%m-%d') for d in dates]
                    filtered_data = monthly_data[monthly_data['Date'].isin(target_dates)]
                    
                    if not filtered_data.empty:
                        # 插入數據
                        inserted = self.insert_stock_data(stock_table, filtered_data)
                        total_inserted += inserted
                        logger.info(f"成功插入 {inserted} 筆 {year}-{month} 數據")
                    else:
                        logger.warning(f"未找到 {year}-{month} 月的目標日期數據")
            
            logger.info(f"{stock_table}: 共補充 {total_inserted} 筆數據")
            return total_inserted
            
        except Exception as e:
            logger.error(f"處理 {stock_table} 時出錯: {str(e)}")
            return 0

    def process_tables_parallel(self, tables, start_date, max_workers=5):
        """並行處理多個表"""
        total_inserted = 0
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # 提交所有處理任務
            future_to_table = {executor.submit(self.process_stock_table, table, start_date): table for table in tables}
            completed = 0
            
            # 獲取結果
            for future in concurrent.futures.as_completed(future_to_table):
                table = future_to_table[future]
                try:
                    inserted = future.result()
                    total_inserted += inserted
                except Exception as e:
                    logger.error(f"處理表 {table} 時出錯: {str(e)}")
                
                completed += 1
                progress = (completed / len(tables)) * 100
                print(f"總進度: {progress:.1f}% ({completed}/{len(tables)})")
        
        return total_inserted

def main():
    start_time = time.time()
    
    # 設定開始日期
    start_date = "2025-02-11"
    
    print(f"開始以最快速度更新股票數據 (從 {start_date} 到今天)")
    logger.info(f"開始以最快速度更新股票數據 (從 {start_date} 到今天)")
    
    updater = FastStockUpdater()
    all_tables = updater.get_stock_tables()
    
    if not all_tables:
        print("未找到股票表")
        return
    
    print(f"找到 {len(all_tables)} 個表格，檢查哪些表格有缺失數據...")
    logger.info(f"找到 {len(all_tables)} 個表格，檢查哪些表格有缺失數據...")
    
    # 篩選出有缺失數據的表格
    tables_with_missing_data = []
    for table in all_tables:
        try:
            missing_dates = updater.check_missing_data(table, start_date)
            if missing_dates:
                tables_with_missing_data.append(table)
                print(f"表格 {table} 缺失 {len(missing_dates)} 筆數據")
        except Exception as e:
            logger.warning(f"檢查表格 {table} 時出錯: {str(e)}")
    
    if not tables_with_missing_data:
        print("沒有表格需要更新，所有數據都是最新的")
        return
    
    print(f"\n找到 {len(tables_with_missing_data)} 個有缺失數據的表格，開始更新...")
    logger.info(f"找到 {len(tables_with_missing_data)} 個有缺失數據的表格，開始更新...")
    
    # 使用並行處理加速，只處理有缺失數據的表格
    print("\n開始並行處理...")
    total_inserted = updater.process_tables_parallel(tables_with_missing_data, start_date, max_workers=8)
    
    end_time = time.time()
    elapsed = end_time - start_time
    
    print(f"\n處理完成! 共補充 {total_inserted} 筆數據")
    print(f"處理時間: {elapsed:.2f} 秒")
    logger.info(f"處理完成! 共補充 {total_inserted} 筆數據, 耗時: {elapsed:.2f} 秒")

if __name__ == "__main__":
    main()

# 另存為 analyze_5880.py
analyze_5880 = {
    'cell_type': 'code',
    'metadata': {
        'id': 'analyze_5880',
        'language': 'python'
    },
    'source': [
        "# 分析合庫金(5880)股票",
        "try:",
        "    conn = connect_to_database()",
        "    if conn is None:",
        "        print(\"無法連接到資料庫\")",
        "    else:",
        "        try:",
        "            # 讀取合庫金股票數據",
        "            query = \"\"\"",
        "            SELECT [Date] as 交易日期,",
        "                   [Open] as 開盤價,",
        "                   [High] as 最高價,",
        "                   [Low] as 最低價,",
        "                   [Close] as 收盤價,",
        "                   [Volume] as 成交量",
        "            FROM [stock].[dbo].[5880TW合庫金]",
        "            ORDER BY [Date]",
        "            \"\"\"",
        "            ",
        "            df_5880 = pd.read_sql(query, conn)",
        "            ",
        "            # 添加股票代碼和名稱列",
        "            df_5880['股票代碼'] = '5880'",
        "            df_5880['股票名稱'] = '合庫金'",
        "            ",
        "            print(\"\n合庫金(5880)基本統計資訊:\")",
        "            print(f\"資料期間: {df_5880['交易日期'].min()} 到 {df_5880['交易日期'].max()}\")",
        "            print(f\"總交易天數: {len(df_5880)}\")",
        "            print(f\"最高價: {df_5880['最高價'].max():.2f}\")",
        "            print(f\"最低價: {df_5880['最低價'].min():.2f}\")",
        "            print(f\"平均收盤價: {df_5880['收盤價'].mean():.2f}\")",
        "            ",
        "            # 進行股價區間分析",
        "            interval_result = price_interval_analysis(df_5880, '5880', m=5)",
        "            if interval_result is not None:",
        "                print(\"\n股價區間分析:\")",
        "                print(interval_result)",
        "            ",
        "            # 進行股價趨勢分析",
        "            trend_result = analyze_stock_trend(df_5880, '5880')",
        "            if trend_result is not None:",
        "                # 計算年度收益率",
        "                annual_returns = calculate_annual_returns(trend_result)",
        "                print(\"\n年度收益率:\")",
        "                print(annual_returns)",
        "                ",
        "                # 計算波動率",
        "                daily_returns = trend_result['收盤價'].pct_change()",
        "                volatility = daily_returns.std() * np.sqrt(252) * 100  # 年化波動率",
        "                print(f\"\n年化波動率: {volatility:.2f}%\")",
        "                ",
        "        except Exception as e:",
        "            print(f\"分析過程中出錯: {e}\")",
        "        finally:",
        "            conn.close()",
        "            print(\"\n資料庫連接已關閉\")",
        "except Exception as e:",
        "    print(f\"執行分析時發生錯誤: {e}\")"
    ]
}

開始以最快速度更新股票數據 (從 2025-02-11 到今天)
找到 1276 個表格，檢查哪些表格有缺失數據...
表格 5533TW皇鼎 缺失 116 筆數據
表格 00652TW 缺失 116 筆數據
表格 2610TW華航 缺失 116 筆數據
表格 9912TW偉聯 缺失 116 筆數據
表格 2458TW義隆 缺失 116 筆數據
表格 2360TW致茂 缺失 116 筆數據
表格 6753TW龍德造船 缺失 116 筆數據
表格 3048TW益登 缺失 116 筆數據
表格 2059TW川湖 缺失 116 筆數據
表格 6283TW淳安 缺失 116 筆數據
表格 1713TW國化 缺失 116 筆數據
表格 1472TW三洋實業 缺失 116 筆數據
表格 4583TW台灣精銳 缺失 116 筆數據
表格 1304TW台聚 缺失 116 筆數據
表格 3583TW辛耘 缺失 116 筆數據
表格 6742TW澤米 缺失 116 筆數據
表格 00956TW 缺失 116 筆數據
表格 6288TW聯嘉 缺失 116 筆數據
表格 00783TW 缺失 116 筆數據
表格 2888ATW 缺失 116 筆數據
表格 00653LTW 缺失 116 筆數據
表格 2611TW志信 缺失 116 筆數據
表格 9911TW櫻花 缺失 116 筆數據
表格 2459TW敦吉 缺失 116 筆數據
表格 8081TW致新 缺失 116 筆數據
表格 2362TW藍天 缺失 116 筆數據
表格 6743TW安普新 缺失 116 筆數據
表格 2062TW橋椿 缺失 116 筆數據
表格 1714TW和桐 缺失 116 筆數據
表格 1473TW台南 缺失 116 筆數據
表格 4588TW玖鼎電力 缺失 116 筆數據
表格 2460TW建通 缺失 116 筆數據
表格 1305TW華夏 缺失 116 筆數據
表格 3591TW艾笛森 缺失 116 筆數據
表格 00954TW 缺失 116 筆數據
表格 3049TW精金 缺失 116 筆數據
表格 00830TW 缺失 116 筆數據
表格 2888BTW 缺失 116 筆數據
表格 00654RTW 缺失 116 筆數據
表格 2612TW中航 缺失 116 筆數據
表格 9914TW美利達 缺失 1

In [2]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
